In [ ]:
#from https://developers.refinitiv.com/en/article-catalog/article/tensorflow-variational-quantum-neural-networks-in-finance
import os
import yfinance as yf
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from tensor_layer_expermental import QuantumDense
import datetime as date
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame
from sklearn.preprocessing import MinMaxScaler


APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL, APCA_API_NEWS_URL = "", "", "", ""

pd.set_option('display.max_columns', None)

ticker = input("Enter stocks to analyze: ")



    
now = date.date.today()
today = now.strftime('%Y-%m-%d')
yesterday = (now - pd.Timedelta('1day')).strftime('%Y-%m-%d')
ninedaysago = (now - pd.Timedelta('9day')).strftime('%Y-%m-%d')

def set_vars(secrets_file="secrets-alpaca.env"):
    """
    This function initializes the credentials by setting APCA_API_KEY_ID, APCA_API_SECRET_KEY, and APCA_API_BASE_URL
    global variables from a specified .env file in the same directory

    Parameters
    ----------
    secrets_file: Default set to secrets-alpaca.env, can be any secrets file containing the

    Returns
    -------
    None

    """

    global APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL

    with open(f"{secrets_file}", 'r') as file:
        contents = file.read()
        env_vars = contents.replace('export ', '').split("\n")
        APCA_API_KEY_ID = env_vars[0].split("=")[1]
        APCA_API_SECRET_KEY = env_vars[1].split("=")[1]
        APCA_API_BASE_URL = env_vars[2].split("=")[1]
        APCA_API_NEWS_URL = env_vars[2].split("=")[1]


def primaryfunc():
    global APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL
    set_vars()
    api = tradeapi.REST(APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL)
    #test1 = api.get_aggs("INTC", 1, 1,'2023-03-21','2023-03-31')
    df = api.get_bars(ticker, TimeFrame.Hour, "2023-03-31", "2023-03-31", adjustment='raw').df
    df['RETURNS'] = df['close'].pct_change()
    df['PRX_MA_ND'] = df['close'].rolling(window=5).mean()
    df['VOLATILITY'] = df['close'].rolling(window=5).std()
    df['TP1_RETURNS'] = df['RETURNS'].shift(-1)
    df['VOLUME'] = df['volume']

    df.dropna(inplace=True)

    df_x = df[['RETURNS', 'VOLUME', 'PRX_MA_ND', 'VOLATILITY']]
    df_y = df['TP1_RETURNS']

    df_x_scaler = MinMaxScaler().fit(df_x)

    forward_test_date = '2024-03-01'

    fdf_x = df_x.loc[forward_test_date:]
    fdf_y = df_y.loc[forward_test_date:]
    df_x = df_x.loc[:forward_test_date]
    df_y = df_y.loc[:forward_test_date]

    df_x_scaled = pd.DataFrame(df_x_scaler.transform(df_x))
    fdf_x_scaled = pd.DataFrame(df_x_scaler.transform(fdf_x))

    x_train, x_test, y_train, y_test = train_test_split(df_x_scaled,
                                                        df_y,
                                                        test_size=0.25,
                                                        random_state=42)
primaryfunc()




In [ ]:
#from https://developers.refinitiv.com/en/article-catalog/article/tensorflow-variational-quantum-neural-networks-in-finance
import os
import yfinance as yf
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from tensor_layer_expermental import QuantumDense
import datetime as date
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame

APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL, APCA_API_NEWS_URL = "", "", "", ""

pd.set_option('display.max_columns', None)

ticker = "INTC"



    
now = date.date.today()
today = now.strftime('%Y-%m-%d')
yesterday = (now - pd.Timedelta('1day')).strftime('%Y-%m-%d')
ninedaysago = (now - pd.Timedelta('9day')).strftime('%Y-%m-%d')

def set_vars(secrets_file="secrets-alpaca.env"):
    """
    This function initializes the credentials by setting APCA_API_KEY_ID, APCA_API_SECRET_KEY, and APCA_API_BASE_URL
    global variables from a specified .env file in the same directory

    Parameters
    ----------
    secrets_file: Default set to secrets-alpaca.env, can be any secrets file containing the

    Returns
    -------
    None

    """

    global APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL

    with open(f"{secrets_file}", 'r') as file:
        contents = file.read()
        env_vars = contents.replace('export ', '').split("\n")
        APCA_API_KEY_ID = env_vars[0].split("=")[1]
        APCA_API_SECRET_KEY = env_vars[1].split("=")[1]
        APCA_API_BASE_URL = env_vars[2].split("=")[1]
        APCA_API_NEWS_URL = env_vars[2].split("=")[1]


def primaryfunc():
    global APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL
    set_vars()
    api = tradeapi.REST(APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL)
    #test1 = api.get_aggs("INTC", 1, 1,'2023-03-21','2023-03-31')
    df = api.get_bars(ticker, TimeFrame.Hour, "2023-03-31", "2023-03-31", adjustment='raw').df
    df['RETURNS'] = df['close'].pct_change()
    df['PRX_MA_ND'] = df['close'].rolling(window=5).mean()
    df['VOLATILITY'] = df['close'].rolling(window=5).std()
    df['TP1_RETURNS'] = df['RETURNS'].shift(-1)

    df.dropna(inplace=True)
    print(df.index)
    
primaryfunc()


